In [1]:
%pip install qiskit==1.2.4
%pip install qiskit-aer==0.15.1
%pip install pylatexenc==2.10


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_gate
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Operator
from qiskit.quantum_info import Statevector
from qiskit import transpile 
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import ControlledGate
import math 

# The aim of the assignment is to simulate the Ekert91 key distribution protocol.

# This notebook is for a simulation of the protocol with an attacker, to demonstrate that the attacker can be detected.

In [3]:
import random
#need to define W and V:
root2 = math.sqrt(2)
root2 = math.sqrt(2)

denom1 = math.sqrt(4 + 2*root2)
denom2 = math.sqrt(4 - 2*root2) 

#ssh - 2667501P


W = [ [ -1 / denom1 , (1 + root2) / denom1 ],
    [  1 / denom2 , (root2 - 1) / denom2 ] ]


V = [ [  1 / denom1 , (1 + root2) / denom1 ],
    [ -1 / denom2 , (root2 - 1) / denom2 ] ]

random_matrix = [ [math.sqrt(1/3), math.sqrt(2/3)],
                  [math.sqrt(2/3), -math.sqrt(1/3)] ]

W_Operator = Operator(W)
V_Operator = Operator(V)
random_Operator = Operator(random_matrix)

X_Cross_W = 0
X_Cross_V = 0
Z_Cross_W = 0
Z_Cross_V = 0

X_Cross_W_array = []
X_Cross_V_array = []
Z_Cross_W_array = []
Z_Cross_V_array = []




#quantum randomness
def random_choice_Alice():
    rqc = QuantumCircuit(1,1)
    rqc.unitary(random_Operator,0)
    rqc.measure(0,0)

    backend = BasicSimulator()
    compiled = transpile(rqc, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    measurement = list(counts.keys())[0][-1]

    if measurement == "0":
        op = "X"

    elif measurement == "1":
        rqc_2 = QuantumCircuit(1,1)
        #equal chance
        rqc_2.h(0)
        rqc_2.measure(0,0)

        backend = BasicSimulator()
        compiled = transpile(rqc_2, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(compiled)
        measurement2 = list(counts.keys())[0][-1]

        if measurement2 == "0":
            op = "W"
        
        elif measurement2 == "1":
            op = "Z"

    return op
    
       

def random_choice_Bob():
    rqc = QuantumCircuit(1,1)
    rqc.unitary(random_Operator,0)
    rqc.measure(0,0)

    backend = BasicSimulator()
    compiled = transpile(rqc, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)
    measurement = list(counts.keys())[0][-1]

    if measurement == "0":
        op = "W"

    elif measurement == "1":
        rqc_2 = QuantumCircuit(1,1)
        #equal chance
        rqc_2.h(0)
        rqc_2.measure(0,0)

        backend = BasicSimulator()
        compiled = transpile(rqc_2, backend)
        job_sim = backend.run(compiled, shots=1)
        result_sim = job_sim.result()
        counts = result_sim.get_counts(compiled)
        measurement2 = list(counts.keys())[0][-1]

        if measurement2 == "0":
            op = "Z"
        
        elif measurement2 == "1":
            op = "V"
       
    return op
        
def entanglePair():
    #add classical bit for eve to save her measurment
    q = QuantumCircuit(2,3) 
    q.h(0)
    q.cx(0,1)
    q.x(1)
    q.z(1)
    return q

def take_average(counts):
    count00 = counts.get("00", 0)
    count01 = counts.get("01", 0)
    count10 = counts.get("10", 0)
    count11 = counts.get("11", 0)
    return  (count00 - count01 - count10 + count11)
   

N = 50

measurements = {}
shared_key_A = []
shared_key_B = []




for i in range(0,(int((9*N)/2))):
    # alice and bob each recieve entagled pair 
    q = entanglePair()

    #bob flips his bits

    q.x(1)
    q.z(1)

    # alice and bob pick their operators 

    alice_operator = random_choice_Alice()
   
    

    bob_operator = random_choice_Bob()

    #eve attacks
    q.measure(0,2)


    # alice measure operator Ai on her qubit
    if alice_operator == "X":
        q.h(0)
        q.measure(0,0)
    elif alice_operator == "W":
        q.unitary(W_Operator, [0])
        q.measure(0,0)
    elif alice_operator == "Z":
        q.measure(0,0)
    
    

    #Bob measures Bj on his qubit
    if bob_operator == "W":
        q.unitary(W_Operator, [1])
        q.measure(1,1)
    elif bob_operator == "Z":
        q.measure(1,1)
    elif bob_operator == "V":
        q.unitary(V_Operator, [1])
        q.measure(1,1)

    backend = BasicSimulator()
    compiled = transpile(q, backend)
    job_sim = backend.run(compiled, shots=1)
    result_sim = job_sim.result()
    counts = result_sim.get_counts(compiled)

    #store measurements for bells inequality but only for the 7/9 of the time where they wont be opposite

    for key, value in counts.items():
        measurements[key] = measurements.get(key, 0) + value


    for result in counts.keys():
        alice_bit = int(result[0])
        bob_bit = int(result[1])


        if ((alice_operator == "W" and bob_operator == "W") or (alice_operator == "Z" and bob_operator == "Z")):
            shared_key_A.append(alice_bit)
            shared_key_B.append(bob_bit)
            
        elif(alice_operator == "X" and bob_operator == "V"):
            if alice_bit == 0:
                alice_bit = -1
            if bob_bit == 0:
                bob_bit = -1
                
            X_Cross_V_array.append(alice_bit*bob_bit)
            X_Cross_V += 1
            
        elif(alice_operator == "X" and bob_operator == "W"):
            if alice_bit == 0:
                alice_bit = -1
            if bob_bit == 0:
                bob_bit = -1
            X_Cross_W_array.append(alice_bit*bob_bit)
            X_Cross_W += 1

        elif(alice_operator == "Z" and bob_operator == "W"):
            if alice_bit == 0:
                alice_bit = -1
            if bob_bit == 0:
                bob_bit = -1
                
            Z_Cross_W_array.append(alice_bit*bob_bit)
            Z_Cross_W += 1

        elif(alice_operator == "Z" and bob_operator == "V"):
            if alice_bit == 0:
                alice_bit = -1
            if bob_bit == 0:
                bob_bit = -1
                
            Z_Cross_V_array.append(alice_bit*bob_bit)
            Z_Cross_V += 1
        

total_shots = sum(measurements.values())
#bells_inequlity = take_average(measurements, total_shots)


#calcualte average for each case
X_Cross_V_Average = 0
for entry in X_Cross_V_array:
    X_Cross_V_Average += entry
X_Cross_V_Average = X_Cross_V_Average/X_Cross_V

X_Cross_W_Average = 0
for entry in X_Cross_W_array:
    X_Cross_W_Average += entry
X_Cross_W_Average = X_Cross_W_Average/X_Cross_W

Z_Cross_W_Average = 0
for entry in Z_Cross_W_array:
    Z_Cross_W_Average += entry
Z_Cross_W_Average = Z_Cross_W_Average/Z_Cross_W

Z_Cross_V_Average = 0
for entry in Z_Cross_V_array:
    Z_Cross_V_Average += entry
Z_Cross_V_Average = Z_Cross_V_Average/Z_Cross_V
    
bells_inequlity = abs(X_Cross_W_Average - X_Cross_V_Average + Z_Cross_W_Average + Z_Cross_V_Average)


print(f"bells_inequality: ",bells_inequlity)
print(f"ALice's shared key: ",shared_key_A)
print(f"Bob's shared key: ",shared_key_B)
print(f"Length of shared key: ",len(shared_key_A))

if bells_inequlity <2.0:
    print("it is highly likely there was an attack as the result of the bells inequality test is lower than 2 and there has been a complete loss in quantum correlation")

elif bells_inequlity < 2.5 or bells_inequlity > 3.1:
    print("there may have been an attack as the result of the bells inequality test is not close to 2.8")






bells_inequality:  1.3782732693803474
ALice's shared key:  [0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
Bob's shared key:  [0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
Length of shared key:  47
it is highly likely there was an attack as the result of the bells inequality test is lower than 2 and there has been a complete loss in quantum correlation
